# Regridding Tool

This tool remaps netCDF (.nc) ice sheet models from one user selected kilometer grid to another.


netCDF Ice Sheet Model File Naming Convention:

example file name: acabf_GrIS_IMAU_IMAUICE1_init.nc

acabf -- 2D variable name<br />
GrIS -- Ice Sheet Name<br />
IMAU -- Modeling group name<br />
IMAUICE1 -- Ice sheet model name<br />
init -- Experiment name


All directories utilized by this tool are located on the CCR Cluster
Location of ISMIP6 netCDF Files - /projects/grid/ghub/ISMIP6/Projections
Location of Grid Mapping Files  - /projects/grid/ghub/ISMIP6/Grids

Subdirectories follow the netCDF ice sheet model file naming convention (excluding 2D variable names):
Location of ISMIP6 netCDF Files - /projects/grid/ghub/ISMIP6/Projections/Ice_Sheet/output/Modeling_Group/Ice_Sheet_Model/experiment/netCDF_file.nc
Location of Grid Mapping Files  - /projects/grid/ghub/ISMIP6/Grids/Ice_Sheet/GDFs/grid_file.nc


To use this tool: 
1. Make sure your desired netCDF files are located on the CCR cluster in the appropriate directory
2. First choose whether you would like to regrid by file or by experiment name. (file lets you select individual files while experiment regrids entire experiment groups)
3. Choose Desired Ice Sheet (Greenland or Antarctic), Modeling Group, Ice Sheet Model, and Experiment
4. Select the netCDF file (or files) you want to regrid
5. Select an ingrid file with the same grid dimension as your currently selected netCDF files
6. Select an outgrid file with the same grid dimension you wish to remap your files
7. Click the "Regrid Files" button to begin regridding process
8. Retrieve your reformatted files from the specified output directory


Regridded Files are output to directory:
/gpfs/scratch/ghub/regrid-tool

Output files can be accessed using Globus:<br />
Globus Endpoint Name: GHub-scratch-regrid-tool<br />
Globus Website    --   https://app.globus.org<br />
Globus Help Docs  --   https://docs.globus.org/how-to/get-started/<br />
UB CCR Help Docs  --   https://ubccr.freshdesk.com/support/solutions/articles/13000071584-connecting-to-ccr-endpoint-and-browsing-your-files<br />

The name of your specific output subdirectory will be printed along with the results of your regridding job when the process is complete.
Output directory format: date_job#_ingrid2outgrid


Background:
- This Jupyter-based tool uses Python 3 with the cdo package for regridding processes
- This tool is compatible with both Greenland(GIS) and Antarctic(AIS) Ice Sheet Data

Notice:
Data used by this tool is raw and may be subject to minor errors.

See Links below for more info about regridding with CDO:<br />
http://www.climate-cryosphere.org/wiki/index.php?title=Regridding<br />
http://www.climate-cryosphere.org/wiki/index.php?title=Regridding_with_CDO

# Select Kernel Restart & Run All to initialize the notebook

In [1]:
import sys,os,json,glob
import math
import getpass
import platform
import subprocess
import shutil
import os
from os import listdir
from os.path import isfile, join
import ipywidgets as widgets
import hublib.ui as ui
import time
from datetime import datetime

from ipywidgets import HBox

from IPython.display import HTML, display

import hublib.use
%use anaconda-6

<IPython.core.display.Javascript object>

# Click the Hide / Show Code Cells button to hide / show the code cells

In [2]:
hideCodeButton = hublib.ui.HideCodeButton(style='success')
display(hideCodeButton)

Button(button_style='success', description='Hide Code Cells', style=ButtonStyle())

In [3]:
#create variables for general info
self_tooldir = os.path.realpath("")

self_srcdir = os.path.join(self_tooldir,"bin")

self_datadir = os.path.join(self_tooldir,"data")

self_workingdir = os.getcwd()

self_homedir = os.path.expanduser("~")

self_rundir = ""

self_user = getpass.getuser()

sys.path.insert(1,self_srcdir)

In [4]:
#list of scalar variables to exclude (cannot be regridded only 2D variables)
invalid_vars = ['lim','limnsw','iareagr','iareafl','tendacabf','tendlibmassbf','tendlibmassbffl','tendlicalvf','tendlifmassbf','tendligroundf']
#function checks if file is scalar
def check_for_scalars(file_name):
    for name in invalid_vars:
        if name in file_name:
            return True
    return False

#function returns number of files user wants regridded
def num_files():
    if regrid_choice.value == 'File':
        return len(file_selection.value)
    if regrid_choice.value == 'Experiment':
        file_count = 0
        if is_choice.value == 'Greenland':
            is_path = '/GrIS/output'
        else:
            is_path = '/AIS/output'
        if type(grp_choice.value) is str:
            mods_path = is_path+"/"+grp_choice.value
            if type(mod_choice.value) is str:
                exp_path = mods_path+"/"+mod_choice.value
                for exp in exp_selection.value:
                    if type(exp) is str:
                        files_path = exp_path+"/"+exp
                        file_count = file_count+len(remove_scalars(get_path_opts(files_path)))
        return file_count

#function calculates wall time based on the smallest grid dimension selected by the user
#the smaller the grid size the more walltime required for regridding
def wall_time():
    if target_location.value=='CCR':
        if ingrid_choice.value=='0.5km' or outgrid_choice.value=='0.5km':
            in_val = 1
            out_val = 1
        else:
            in_val = ingrid_choice.value
            in_val = int(in_val.replace('km',''))
            out_val = outgrid_choice.value
            out_val = int(out_val.replace('km',''))
    else:
        if thule_ingrid_choice.value=='0.5km' or thule_outgrid_choice.value=='0.5km':
            in_val = 1
            out_val = 1
        else:
            in_val = thule_ingrid_choice.value
            in_val = int(in_val.replace('km',''))
            out_val = thule_outgrid_choice.value
            out_val = int(out_val.replace('km',''))
    small_grid = 0
    if out_val > in_val:
        small_grid = in_val
    else:
        small_grid = out_val
    if small_grid > 20:
        return 15
    elif small_grid > 5:
        return 30
    elif small_grid > 3:
        return 45
    else:
        return 60
        
          
#function generates parameter file, 'arguments.csv'
#file is passed with submit to execute parameter sweep
def gen_param_file():
    param_file = open("./bin/arguments.csv","w")
    file_num = num_files()
    if regrid_choice.value == 'File':
        param_file.write("@@ice_sheet,@@mod_grp,@@is_model,@@exp,@@ingrid,@@outgrid,@@file_name,@@num_files\n")
        for f in file_selection.value:
            if is_choice.value=='Greenland':
                param_file.write("GIS,")
            else:
                param_file.write("AIS,")
            param_file.write(grp_choice.value+',')
            param_file.write(mod_choice.value+',')
            param_file.write(exp_choice.value+',')
            param_file.write(ingrid_choice.value+',')
            param_file.write(outgrid_choice.value+',')
            param_file.write(f+',')
            param_file.write(str(file_num))
            param_file.write('\n')
    if regrid_choice.value == 'Experiment':
        param_file.write("@@ice_sheet,@@mod_grp,@@is_model,@@exp,@@ingrid,@@outgrid,@@file_name,@@num_files\n")
        if is_choice.value == 'Greenland':
            is_path = '/GrIS/output'
        else:
            is_path = '/AIS/output'
        grp_path = is_path+'/'+grp_choice.value
        mod_path = grp_path+'/'+mod_choice.value
        for exp in exp_selection.value:
            exp_path = mod_path+'/'+exp
            files = get_path_opts(exp_path)
            for file in files:
                if check_for_scalars(file):
                    continue
                if is_choice.value=='Greenland':
                    param_file.write("GIS,")
                else:
                    param_file.write("AIS,")
                param_file.write(grp_choice.value+',')
                param_file.write(mod_choice.value+',')
                param_file.write(exp+',')
                param_file.write(ingrid_choice.value+',')
                param_file.write(outgrid_choice.value+',')
                param_file.write(file+',')
                param_file.write(str(file_num))
                param_file.write('\n')   
    !chmod a+r ./bin/arguments.csv
    param_file.close()
    
#print out to give info to user as regrid job begins
def workflow_start_display():
    data_dir = '/projects/grid/ghub/ISMIP6/Projections'
    files = []
    task_num = 0
    a = open('./bin/arguments.csv')
    for line in a:
        if '@@' in line:
            continue
        files.append(line.split(',')[6])
        task_num = task_num+1
    a.close()
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    with summary_output:
        print('Regridding in progress...\n\n')
        print('tooldir: '+self_tooldir)
        print('srcdir: '+self_srcdir)
        print('datadir: '+data_dir)
        print('workingdir: '+self_workingdir)
        print('\nJob Start Time: '+current_time)
        print('Job Wall Time: '+str(wall_time())+' minutes')
        print('NOTICE: Job May Require More Than Full Wall Time to Complete!')
        print('You will be emailed upon completion of this job')
        print('\nNumber of Files: '+str(task_num))
        print('\nIce Sheet: '+is_choice.value)
        print('Modeling Group: '+grp_choice.value)
        print('Ice Sheet Model: '+mod_choice.value)
        if regrid_choice.value=='File':
            print('Experiment Group: '+exp_choice.value)
            print('\nProcessing Files:')
            for name in files:
                print(name)
        else:
            print('\nExperiment Groups: ')
            for exp in exp_selection.value:
                print(exp)
        print('\nIngrid: '+ingrid_choice.value+' --> '+'Outgrid: '+outgrid_choice.value)

#retrieves most recent job number from term_out.txt file
def retrieve_job_num():
    f = open('term_out.txt','r')
    job_num = 0
    for line in f:
        try:
            dets = line.split(' ')
            job_num = dets[1]
            ext = 8-len(job_num)
            for i in range(ext):
                job_num = '0'+job_num
            return job_num
        except:
            return job_num

#finds most recent job directory
def job_dir():
    stream = os.popen('find * -type d -cmin -50')
    output = stream.read()
    most_rec_dir = ""
    recent_dirs = output.split('\n')
    recent_dirs = list(filter(None,recent_dirs))
    for i in range(2,len(recent_dirs)):
        most_rec_dir = recent_dirs[len(recent_dirs)-i]
        if '/' in most_rec_dir:
            continue
        else:
            break
    most_rec_dir = "./"+most_rec_dir
    return most_rec_dir

#creates directory in user's home directory to store job log files
def create_localdir():
    job_num = retrieve_job_num()
    local_dir = self_homedir+'/regrid_job'+str(job_num)+'_info'
    os.mkdir(local_dir)
    return local_dir

#creates log file job_summary.txt
#also prints job summary to tool interface
total_failure = False
failed = False
fname = ''
passed_files = []
def job_summary(dir_name, elapsed_time, local_dir):
    write_file = local_dir+'/job_summary.txt'
    of = open(write_file,'w')
    if target_location.value=='CCR':
        data_dir = '/projects/grid/ghub/ISMIP6/Projections'
    else:
        data_dir = '/gpfs/scratch/ghub/regrid-tool/thule-regrid-targets'
    passed_files = []
    failed_files = []
    error_message = []
    time_limit_reached = False
    for subdir, dirs, files in os.walk(dir_name):
        failed = False
        fname = ''
        for file in files:
            if 'slurm' in file:
                time_limit_reached = True
                continue
            if '.stderr' in file:
                f = open(os.path.join(subdir,file),'r')
                for line in f:
                    if target_location.value=='CCR':
                        if '/projects/grid/ghub/ISMIP6/Projections' in line:
                            fname = (line.split('/')[19]).split(' ')[0]
                            break
                    else:
                        if '/gpfs/scratch/ghub/regrid-tool/thule-regrid-targets' in line:
                            fname = (line.split('/')[15]).split(' ')[0]
                            break
                f.close()
                f = open(os.path.join(subdir,file),'r')
                for line in f:
                    if 'File size limit exceeded' in line:
                        #failed_files.append(fname)
                        error_message.append('File cannot be remapped to chosen outgrid dimension.')
                        failed = True
                        break
                    if 'Unsupported grid type' in line:
                        #failed_files.append(fname)
                        error_message.append('Unsupported Grid Type')
                        failed = True
                        break
                    if 'Grid cell corner coordinates missing!' in line:
                        error_message.append('File ingrid dimension is incorrect.')
                        failed = True
                        break
                    if 'Segmentation fault' in line:
                        error_message.append('File ingrid or outgrid dimension is incorrect.')
                        failed = True
                        break
                    if 'No grid with' in line:
                        error_message.append('File could not be remapped to chosen outgrid.')
                        failed = True
                        break
                    if 'Open failed' in line:
                        error_message.append('Failed to open grid mapping file')
                        failed = True
                        break
                    if 'failed' in line:
                        error_message.append('Remapping File Failed')
                        failed = True
                        break
                    if 'error' in line:
                        #failed_files.append(fname)
                        error_message.append('Remapping could not be completed')
                        time_limit_reached = True
                        failed = True
                        break
                    if 'Killed' in line:
                        #failed_files.append(fname)
                        error_message.append('Remapping could not be completed')
                        failed = True
                        break
                    if 'Abort' in line:
                        error_message.append('Remapping could not be completed')
                f.close()
                if failed == False:
                    passed_files.append(fname)
                else:
                    failed_files.append(fname)
    passed_files = list(filter(lambda x: x != "", passed_files))
    of.write('Job Summary\n\n')
    of.write('\ntooldir: '+self_tooldir)
    of.write('\nsrcdir: '+self_srcdir)
    of.write('\ndatadir: '+data_dir)
    of.write('\nworkingdir: '+self_workingdir)
    of.write('\n\nElapsed Time:\n')
    of.write(str(elapsed_time) + " seconds\n")
    of.write(str(elapsed_time/60.0) + " minutes\n")
    of.write(str(elapsed_time/3600.0) + " hours\n")
    of.write('Includes Wait Time!')
    of.write('\nFiles Processed: '+str(len(passed_files)))
    of.write('\nFiles Failed: '+str(len(failed_files)))
    if target_location.value == 'CCR':
        of.write('\n\nIce Sheet: '+is_choice.value)
        of.write('\nModeling Group: '+grp_choice.value)
        of.write('\nIce Sheet Model: '+mod_choice.value)
    with summary_output:
        print('Job Summary\n\n')
        print('tooldir: '+self_tooldir)
        print('srcdir: '+self_srcdir)
        print('datadir: '+data_dir)
        print('workingdir: '+self_workingdir)
        print('\n\nElapsed Time:')
        print(str("{:.2f}".format(elapsed_time)) + " seconds")
        print(str("{:.2f}".format(elapsed_time/60.0)) + " minutes")
        print(str("{:.2f}".format(elapsed_time/3600.0)) + " hours")
        print('Includes Wait Time!')
        print('\nFiles Processed: '+str(len(passed_files)))
        print('Files Failed: '+str(len(failed_files)))
        if target_location.value=='CCR':
            print('\nIce Sheet: '+is_choice.value)
            print('Modeling Group: '+grp_choice.value)
            print('Ice Sheet Model: '+mod_choice.value)
    if regrid_choice.value=='File':
        if target_location.value=='CCR':
            of.write('\nExperiment Group: '+exp_choice.value)
            of.write('\nIngrid: '+ingrid_choice.value+' --> '+'Outgrid: '+outgrid_choice.value)
        else:
            of.write('\nIngrid: '+thule_ingrid_choice.value+' --> Outgrid: '+thule_outgrid_choice.value)
        with summary_output:
            if target_location.value=='CCR':
                print('Experiment Group: '+exp_choice.value)
                print('\nIngrid: '+ingrid_choice.value+' --> '+'Outgrid: '+outgrid_choice.value)
            else:
                print('\nIngrid: '+thule_ingrid_choice.value+' --> Outgrid: '+thule_outgrid_choice.value)
        if len(passed_files)>0:
            of.write('\n\nProcessed Files:')
            for f in passed_files:
                of.write(f)
            with summary_output:
                print('\nProcessed Files:')
                for f in passed_files:
                    print(f)
    else:
        if target_location.value=='CCR':
            of.write('\nIngrid: '+ingrid_choice.value+' --> '+'Outgrid: '+outgrid_choice.value)
            of.write('\nExperiment Groups: ')
            for exp in exp_selection.value:
                of.write(exp)
        else: 
            of.write('\nIngrid: '+thule_ingrid_choice.value+' --> '+'Outgrid: '+thule_outgrid_choice.value)
        with summary_output:
            if target_location.value=='CCR':
                print('\nIngrid: '+ingrid_choice.value+' --> '+'Outgrid: '+outgrid_choice.value)
                print('\nExperiment Groups:')
                for exp in exp_selection.value:
                    print(exp)
            else:
                print('\nIngrid: '+thule_ingrid_choice.value+' --> '+'Outgrid: '+thule_outgrid_choice.value)
    if time_limit_reached == True:
        of.write('\n\nERROR: Regrid Reached Max Wall Time:')
        of.write('\nSome files may not have been compressed properly')
        with summary_output:
            print('\nERROR: Regrid Reached Max Wall Time:')
            print('Some files may not have been compressed properly')
    if len(failed_files)>0:
        of.write('\n\nFailed Files:')
        for i in range(len(failed_files)):
            of.write(failed_files[i]+': '+error_message[i])
        with summary_output:
            print('\nFailed Files:')
            for i in range(len(failed_files)):
                print(failed_files[i]+': '+error_message[i])
    of.close()
    if len(passed_files)==0:
        global total_failure
        total_failure = True

#creates log file job_info.txt
def job_info(dir_name, local_dir):
    write_file = local_dir+'/job_info.txt'
    of = open(write_file,'w')
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if 'slurm' in file:
                continue
            if '.stderr' in file:
                f = open(os.path.join(subdir,file),'r')
                for line in f:
                    if target_location.value=='CCR':
                        if '/projects/grid/ghub/ISMIP6/Projections' in line:
                            fname = (line.split('/')[19]).split(' ')[0]
                            of.write(fname)
                            of.write('\n')
                            break
                    else:
                        if '/gpfs/scratch/ghub/regrid-tool/thule-regrid-targets' in line:
                            fname = (line.split('/')[15]).split(' ')[0]
                            of.write(fname)
                            of.write('\n')
                            break
                f.close()
                f = open(os.path.join(subdir,file),'r')
                for line in f:
                    if 'tar' in line:
                        continue
                    if 'gzip' in line:
                        continue
                    if 'mkdir' in line:
                        continue
                    of.write(line)
                f.close()
                of.write('\n')
    of.close()

#gets cluster output directory and returns it
def retrieval_info(dir_name):
    output_file = ''
    for subdir, dirs, files in os.walk(dir_name):
        for file in files:
            if '.stdout' in file:
                f = open(os.path.join(subdir,file),'r')
                for line in f:
                    if '/gpfs/scratch/ghub/regrid-tool' in line:
                        output_file = line
                        break
                f.close()
    return output_file

#prints job output to tool from log file job_info.txt
def print_result(local_dir):
    read_file = local_dir+'/job_info.txt'
    f = open(read_file,'r')
    with job_output:
        print("Regrid Job Result: \n \n")
        for line in f:
            print(line)
    f.close()

In [5]:
#retrieves updated directory structure from regrid job file
def get_opts_list():
    of = open('dir_contents.txt','w')
    rec_jobdir = job_dir()
    for subdir, dirs, files in os.walk(rec_jobdir):
        file_num = 0
        try:
            file_num = int(subdir.split('/')[2])
        except:
            continue
        if file_num == 1:
            for file in files:
                if '.stdout' in file:
                    f = open(os.path.join(subdir,file),'r')
                    for line in f:
                        of.write(line)
                    f.close()
    of.close()

#function moves a passed file to the current working directory
def move_file(file):
    dest_path = self_workingdir
    cmd = 'mv ./'+file+' '+dest_path
    os.system(cmd)

In [6]:
#moves tar file from job to user's home directory
#moved so it can be downloaded using hublib widget
def move_tars():
    job_num = retrieve_job_num()
    tar_archive=job_num+'_archive.tar.gz'
    if os.path.isfile(tar_archive):
        cmd = 'mv '+tar_archive+' '+self_workingdir
        os.system(cmd)
        return True
    else:
        return False

In [7]:
def start_regrid(change):
    if len(file_selection.value)==0 and len(exp_selection.value)==0:
        return
    summary_output.clear_output()
    job_output.clear_output()
    gen_param_file()
    workflow_start_display()
    startTime = time.time()
    submit_cmd = 'submit -v ccr-ghub -w '+str(wall_time())+' -d ./bin/arguments.csv -i ./bin/cron_script.sh ./bin/regrid_script.sh @@ice_sheet @@mod_grp @@is_model @@exp @@ingrid @@outgrid @@file_name @@num_files > term_out.txt'
    with job_output:
        os.system(submit_cmd)
    job_num = retrieve_job_num()
    if job_num==0:
        with job_output:
            print('Error: Job info could not be retrieved')
        return
    archive_name = str(job_num)+'_archive.tar.gz'
    job_dir = './'+str(job_num)
    of_name = retrieval_info(job_dir)
    #tar up files in job directory on cluster
    tar_cmd = 'submit -v ccr-ghub -w '+str(wall_time())+' ./bin/tar_script.sh '+str(job_num)+' '+of_name+' '+str(num_files())
    os.system(tar_cmd)
    elapsed_time = time.time() - startTime
    summary_output.clear_output()
    local_dir = create_localdir()
    job_summary(job_dir,elapsed_time, local_dir)
    job_info(job_dir, local_dir)
    email_subject = 'Regrid Job #'+str(job_num)+' Done'
    email_text = 'Your Regrid Job is Complete!\n'
    email_text = email_text+'Output File:\n'+of_name+'/'+archive_name
    email_text = email_text+'\nOutput files can be accessed using Globus:'
    email_text = email_text+'\nGlobus Endpoint Name: GHub-scratch-regrid-tool'
    email_text = email_text+'\nGlobus Website    --   https://app.globus.org '
    global total_failure
    if total_failure:
        email_text = email_text+'NOTICE: Your Job Failed Completely\nNo Files Regridded Correctly!'
    email_cmd = 'submit --progress silent mail2self -t "'+email_text+'" -s "'+email_subject+'"'
    os.system(email_cmd)
    job_output.clear_output()
    print_result(local_dir)
    write_file = local_dir+'/job_summary.txt'
    of = open(write_file,'a')
    with summary_output:
        of.write('\n\nLog Files Located in Local Directory: '+local_dir)
        of.write('\nRegridded Files Located in Directory: '+of_name)
        of.write('\nArchive File Name: '+archive_name)
        print('\n\nLog Files Located in Local Directory:')
        print(local_dir)
        print('\nRegridded Files Located in Directory: ')
        print(of_name)
        print('Archive File Name:')
        print(archive_name)
    if total_failure:
        with summary_output:
            print('\nJob Failed Completely')
            print('No files regridded correctly')
            total_failure = False
            return
    tar_moved = move_tars()
    if tar_moved:
        tar_name = str(job_num)+'_archive.tar.gz'
        tar_path = self_workingdir
        tar_file = tar_path+'/'+tar_name
        downloadSmallBatch = hublib.ui.Download(tar_name, label='Download Tar', style='success', icon='fa-arrow-circle-down')
        with summary_output:
            print('\nTar File Located in Hub Directory:')
            print(tar_path)
            display(HTML('<h4>Download Tar File: %s</h4>' %tar_name))
            display(downloadSmallBatch)
    else:
        with summary_output:
            of.write('\nOutput files can be accessed using Globus:')
            of.write('\nGlobus Endpoint Name: GHub-scratch-regrid-tool')
            of.write("\nGlobus Website     --   https://app.globus.org")
            of.write('\nGlobus Help Docs   --   https://docs.globus.org/how-to/get-started/')
            of.write('\nCCR Freshdesk Docs --   https://ubccr.freshdesk.com/support/solutions/articles/13000071584-connecting-to-ccr-endpoint-and-browsing-your-files')
            print("\nOutput files can be accessed using Globus:")
            print("Globus Endpoint Name: GHub-scratch-regrid-tool")
            print("Globus Website      --   https://app.globus.org")
            print("Globus Help Docs    --   https://docs.globus.org/how-to/get-started/")
            print("CCR Freshdesk Docs  --   https://ubccr.freshdesk.com/support/solutions/articles/13000071584-connecting-to-ccr-endpoint-and-browsing-your-files")
    of.close()

In [8]:
execute_regrid = widgets.Button(description='Regrid Files',button_style='success')
execute_regrid.on_click(start_regrid)

Thule Regrid

In [9]:
def gen_thule_params():
    of = open('./bin/arguments.csv','w')
    of.write('@@dir_name,@@file_name,@@ingrid,@@outgrid\n')
    for f in thule_file_choice.value:
        of.write(thule_dir_choice.value+','+f+','+thule_ingrid_choice.value+','+thule_outgrid_choice.value+'\n')
    of.close()
    
def thule_start_display():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    with summary_output:
        print('Thule Regridding in progress... ')
        print('\ntooldir: '+self_tooldir)
        print('srcdir: '+self_srcdir)
        print('datadir: /gpfs/scratch/ghub/regrid-tool/thule-regrid-targets')
        print('workingdir: '+self_workingdir)
        print('\nJob Start Time: '+current_time)
        print('Job Wall Time: '+str(wall_time())+' minutes')
        print('Notice: Job May Require More than Full Wall Time to Complete')
        print('You will be emailed upon completion of this job')
        print('\nNumber of Files: '+str(len(thule_file_choice.value)))
        print('\nProcessing Files: ')
        for f in thule_file_choice.value:
            print(f)
        print('\nIngrid: '+thule_ingrid_choice.value+' --> Outgrid: '+thule_outgrid_choice.value)

In [10]:
def exe_thule_regrid(change):
    if len(thule_file_choice.value)==0:
        return
    summary_output.clear_output()
    job_output.clear_output()
    gen_thule_params()
    thule_start_display()
    startTime = time.time()
    submit_cmd = 'submit -v ccr-ghub -w '+str(wall_time())+' -d ./bin/arguments.csv ./bin/thule_regrid.sh @@dir_name @@file_name @@ingrid @@outgrid > term_out.txt'
    os.system(submit_cmd)
    job_num = retrieve_job_num()
    if job_num==0:
        with job_output:
            print('Error: Job info could not be retrieved')
        return
    archive_name = str(job_num)+'_archive.tar.gz'
    job_dir = './'+str(job_num)
    of_name = retrieval_info(job_dir)
    #tar up files in job directory on cluster
    tar_cmd = 'submit -v ccr-ghub -w '+str(wall_time())+' ./bin/tar_script.sh '+str(job_num)+' '+of_name+' '+str(len(thule_file_choice.value))
    os.system(tar_cmd)
    elapsed_time = time.time() - startTime
    summary_output.clear_output()
    local_dir = create_localdir()
    job_summary(job_dir,elapsed_time, local_dir)
    job_info(job_dir, local_dir)
    email_subject = 'Regrid Job #'+str(job_num)+' Done'
    email_text = 'Your Regrid Job is Complete!\n'
    email_text = email_text+'Output File:\n'+of_name+'/'+archive_name
    email_text = email_text+'\nOutput files can be accessed using Globus:'
    email_text = email_text+'\nGlobus Endpoint Name: GHub-scratch-regrid-tool'
    email_text = email_text+'\nGlobus Website    --   https://app.globus.org '
    global total_failure
    if total_failure:
        email_text = email_text+'NOTICE: Your Job Failed Completely\nNo Files Regridded Correctly!'
    email_cmd = 'submit --progress silent mail2self -t "'+email_text+'" -s "'+email_subject+'"'
    os.system(email_cmd)
    job_output.clear_output()
    print_result(local_dir)
    write_file = local_dir+'/job_summary.txt'
    of = open(write_file,'a')
    with summary_output:
        of.write('\n\nLog Files Located in Local Directory: '+local_dir)
        of.write('\nRegridded Files Located in Directory: '+of_name)
        of.write('\nArchive File Name: '+archive_name)
        print('\n\nLog Files Located in Local Directory:')
        print(local_dir)
        print('\nRegridded Files Located in Directory: ')
        print(of_name)
        print('Archive File Name:')
        print(archive_name)
    if total_failure:
        with summary_output:
            print('\nJob Failed Completely')
            print('No files regridded correctly')
            total_failure = False
            return
    tar_moved = move_tars()
    if tar_moved:
        tar_name = str(job_num)+'_archive.tar.gz'
        tar_path = self_workingdir
        tar_file = tar_path+'/'+tar_name
        downloadSmallBatch = hublib.ui.Download(tar_name, label='Download Tar', style='success', icon='fa-arrow-circle-down')
        with summary_output:
            print('\nTar File Located in Hub Directory:')
            print(tar_path)
            display(HTML('<h4>Download Tar File: %s</h4>' %tar_name))
            display(downloadSmallBatch)
    else:
        with summary_output:
            of.write('\nOutput files can be accessed using Globus:')
            of.write('\nGlobus Endpoint Name: GHub-scratch-regrid-tool')
            of.write("\nGlobus Website     --   https://app.globus.org")
            of.write('\nGlobus Help Docs   --   https://docs.globus.org/how-to/get-started/')
            of.write('\nCCR Freshdesk Docs --   https://ubccr.freshdesk.com/support/solutions/articles/13000071584-connecting-to-ccr-endpoint-and-browsing-your-files')
            print("\nOutput files can be accessed using Globus:")
            print("Globus Endpoint Name: GHub-scratch-regrid-tool")
            print("Globus Website      --   https://app.globus.org")
            print("Globus Help Docs    --   https://docs.globus.org/how-to/get-started/")
            print("CCR Freshdesk Docs  --   https://ubccr.freshdesk.com/support/solutions/articles/13000071584-connecting-to-ccr-endpoint-and-browsing-your-files")
    of.close()

# User Input Widgets

In [11]:
def move_to_data(file):
    dest_path = '/data/groups/ghub/tools/regrid'
    cmd = 'mv ./'+file+' '+dest_path
    os.system(cmd)
#initializes options file before running regrid
def initialize_opts_file(change):
    os.system('submit -v ccr-ghub -w 5 ./bin/cron_script.sh > term_out.txt')
    job_num = retrieve_job_num()
    rec_file = './'+job_num+'.stdout'
    f = open(rec_file,'r')
    of = open('dir_contents.txt','w')
    for line in f:
        of.write(line)
    f.close()
    of.close()
    move_to_data('dir_contents.txt')

In [12]:
gen_opts = widgets.Button(description = 'Update File Selection', button_style = 'info')
gen_opts.on_click(initialize_opts_file)

In [13]:
#gets path options from file in /data to update user interface
modelgrps_path = '/projects/grid/ghub/ISMIP6/Projections'
def get_path_opts(path_ext):
    path = modelgrps_path+path_ext
    f = open('/data/groups/ghub/tools/regrid/dir_contents.txt','r')
    lines = f.readlines()
    opts = []
    for i in range(0,len(lines)):
        if path == lines[i].rstrip():
            for j in range(i+1,len(lines)):
                if '/' in lines[j].rstrip():
                    break
                opts.append(lines[j].rstrip())
    return opts

#filters out files from directory listings
def filter_files(opts_list):
    remove_list = []
    for file in opts_list:
        if '.' in file:
            remove_list.append(file)
    for file in remove_list:
        opts_list.remove(file)
    return opts_list

#removes 2D Scalar variables from file list
def remove_scalars(file_list):
    to_remove = []
    for file in file_list:
        varname = file.split('_')[0]
        for name in invalid_vars:
            if name == varname:
                to_remove.append(file)
                break
    for file in to_remove:
        file_list.remove(file)
    return file_list

In [14]:
#handles changes in regrid choice selection File or Experiment
def interface_update(change):
    if regrid_choice.value == 'File':
        file_selection.layout.visibility = "visible"
        exp_choice.layout.visibility = "visible"
        exp_selection.layout.visibility = "hidden"
    if regrid_choice.value == 'Experiment':
        file_selection.layout.visibility = "hidden"
        exp_choice.layout.visibility = "hidden"
        exp_selection.layout.visibility = "visible"

#adjusts dropdowns when ice sheet is switched between antarctica and greenland
def ice_sheet_update(change):
    select_all.value = False
    grp_choice.disabled = True
    ingrid_choice.disabled = True
    outgrid_choice.disabled = True
    if change['new']=='Antarctica':
        grp_choice.options = []
        mod_choice.options = []
        exp_choice.options = []
        exp_selection.options = []
        file_selection.options = []
        is_path = "/AIS/output"
        grps = filter_files(get_path_opts(is_path))
        grp_choice.options = grps
        ingrid_choice.options = ['2km','4km','5km','8km','10km','16km','20km','32km','40km','64km']
        outgrid_choice.options = ['4km','5km','8km','10km','16km','20km','32km','40km','64km']
    if change['new']=='Greenland':
        grp_choice.options = []
        mod_choice.options = []
        exp_choice.options = []
        exp_selection.options = []
        file_selection.options = []
        is_path = "/GrIS/output"
        grps = filter_files(get_path_opts(is_path))
        grp_choice.options = grps
        ingrid_choice.options = ['0.5km','1km','2km','3km','4km','5km','6km','8km','10km','12km','15km','16km','20km','24km','30km','40km']
        outgrid_choice.options = ['1km','2km','3km','4km','5km','6km','8km','10km','12km','15km','16km','20km','24km','30km','40km']
    outgrid_choice.disabled = False
    ingrid_choice.disabled = False
    grp_choice.disabled = False
    
#handles changes to model group selection    
def mod_grp_update(change):
    select_all.value = False
    exp_choice.options = []
    exp_selection.options = []
    file_selection.options = []
    is_path = ""
    if is_choice.value=='Greenland':
        is_path = "/GrIS/output"
    else:
        is_path = "/AIS/output"
    if type(grp_choice.value) is str:
        mods_path = is_path+"/"+grp_choice.value
        mod_choices = filter_files(get_path_opts(mods_path))
        if len(mod_choices)>0 and mod_choices[0].islower():
            mod_choice.options = []
            exp_choice.options = mod_choices
            exp_selection.options = mod_choices
        else:
            mod_choice.options = mod_choices

#handles changes to ice sheet model selection
def ice_mod_update(change):
    select_all.value = False
    exp_choice.options = []
    exp_selection.options = []
    file_selection.options = []
    is_path = ""
    if is_choice.value=='Greenland':
        is_path = "/GrIS/output"
    else:
        is_path = "/AIS/output"
    if type(grp_choice.value) is str:
        mods_path = is_path+"/"+grp_choice.value
        if type(mod_choice.value) is str:
            #exp_path = mods_path+"/"+mod_choice.value
            if mod_choice.value is not None:
                exp_path = mods_path+"/"+mod_choice.value
            else:
                exp_path = mods_path
            exp_choice.options = filter_files(get_path_opts(exp_path))
            exp_selection.options = filter_files(get_path_opts(exp_path))

#function handles when experiment choice is changed
def exp_update(change):
    select_all.value = False
    is_path = ""
    if is_choice.value=='Greenland':
        is_path = "/GrIS/output"
    else:
        is_path = "/AIS/output"
    if type(grp_choice.value) is str:
        mods_path = is_path+"/"+grp_choice.value
        if type(mod_choice.value) is str:
            exp_path = mods_path+"/"+mod_choice.value
            if type(exp_choice.value) is str:
                files_path = exp_path+"/"+exp_choice.value
                file_selection.options = remove_scalars(get_path_opts(files_path))
        elif type(exp_choice.value) is str:
            files_path = mods_path+"/"+exp_choice.value
            file_selection.options = remove_scalars(get_path_opts(files_path))
    if len(exp_choice.options)>0:
        exp_split = exp_choice.options[0].split('_')
        ingrid_val = exp_split[len(exp_split)-1]
        try:
            ingrid_val = int(ingrid_val)
            ingrid_val = str(ingrid_val)+'km'
            ingrid_choice.value = ingrid_val
        except:
            ingrid_val = ''
            
#function to handle when select all widget is clicked            
def all_selected(change):
    if select_all.value is True:
        file_selection.value = file_selection.options
        file_selection.disabled = True
        exp_selection.value = exp_selection.options
        exp_selection.disabled = True
    else:
        file_selection.value = []
        file_selection.disabled = False
        exp_selection.value = []
        exp_selection.disabled = False
                
#function updates outgrid options
#ensures ingrid and outgrid files do not have the same dimension
def adjust_grid_opts(change):
    temp_list = list(ingrid_choice.options)
    temp_list.remove(ingrid_choice.value)
    outgrid_choice.options = temp_list

In [15]:
style = {'description_width':'initial'}
is_choice = widgets.RadioButtons(description = 'Ice Sheet:', options = ['Greenland','Antarctica'], style = style)
is_choice.observe(ice_sheet_update,names='value')

In [16]:
model_grps = filter_files(get_path_opts("/GrIS/output"))
grp_choice = widgets.Dropdown(description = 'Modeling Group',options = model_grps, layout = widgets.Layout(visibility = 'visible'), style = style)
grp_choice.observe(mod_grp_update,names='value')

In [17]:
temp_mods_path = ""
if len(model_grps)>0:
    temp_mods_path = "/GrIS/output/"+model_grps[0]
ice_mods = filter_files(get_path_opts(temp_mods_path))
mod_choice = widgets.Dropdown(description = 'Ice Sheet Model',options = ice_mods, layout = widgets.Layout(visibility = 'visible'), style = style)
mod_choice.observe(ice_mod_update,names='value')

In [18]:
temp_exp_path = ""
if len(ice_mods)>0:
    temp_exp_path = "GrIS/output/"+model_grps[0]+"/"+ice_mods[0]
exp_opts = get_path_opts(temp_exp_path)
exp_choice = widgets.Dropdown(description = 'Experiment:', options = exp_opts, layout = widgets.Layout(visibility = 'visible'), style = style)
exp_choice.observe(exp_update,names='value')

In [19]:
temp_exp_path = ""
if len(ice_mods) > 0:
    temp_exp_path = "GrIS/output/"+model_grps[0]+"/"+ice_mods[0]
exp_opts = get_path_opts(temp_exp_path)
exp_selection = widgets.SelectMultiple(description = "Experiments: ", options = exp_opts, style = style)
exp_selection.layout.visibility = "hidden"

In [20]:
file_choices = []
file_selection = widgets.SelectMultiple(description = 'netCDF Files:', options = file_choices, layout = widgets.Layout(width='20%',height='100%',visibility = 'visible'), style = style)

In [21]:
regrid_choice = widgets.ToggleButtons(description = "Regrid by: ", options = ['Experiment','File'])
regrid_choice.observe(interface_update)
regrid_choice.value = 'File'

In [22]:
select_all = widgets.Checkbox(value = False, description = 'Select All: ')
select_all.observe(all_selected)

In [23]:
ingrid_options = ['0.5km','1km','2km','3km','4km','5km','6km','8km','10km','12km','15km','16km','20km','24km','30km','40km']
ingrid_choice = widgets.Dropdown(description = 'Ingrid Dimension:', options = ingrid_options,layout = widgets.Layout(visibility = 'visible'), style = style)
ingrid_choice.observe(adjust_grid_opts,names='value')

In [24]:
outgrid_options = ['0.5km','1km','2km','3km','4km','5km','6km','8km','10km','12km','15km','16km','20km','24km','30km','40km']
outgrid_options.remove(ingrid_choice.value)
outgrid_choice = widgets.Dropdown(description = 'Outgrid Dimension:', options = outgrid_options, layout = widgets.Layout(visibility = 'visible'), style = style)

In [25]:
job_output = widgets.Output(layout={'border': '1px solid black'})
summary_output = widgets.Output(layout={'border': '1px solid black'})

In [26]:
showmore_button = widgets.Button(description = 'Full Job Info', button_style = 'info')

Thule Widgets

In [27]:
def target_change(change):
    if target_location.value == 'CCR':
        summary_output.clear_output()
        job_output.clear_output()
        gen_opts.layout.visibility = 'visible'
        regrid_choice.layout.visibility = 'visible'
        is_choice.layout.visibility = 'visible'
        grp_choice.layout.visibility = 'visible'
        mod_choice.layout.visibility = 'visible'
        exp_choice.layout.visibility = 'visible'
        select_all.layout.visibility = 'visible'
        file_selection.layout.visibility = 'visible'
        ingrid_choice.layout.visibility = 'visible'
        outgrid_choice.layout.visibility = 'visible'
        execute_regrid.layout.visibility = 'visible'
        regrid_choice.value='File'
        #hide thule widgets
        thule_instruction.layout.visibility = 'hidden'
        check_for_files.layout.visibility = 'hidden'
        thule_dir_choice.layout.visibility = 'hidden'
        thule_select_all.layout.visibility = 'hidden'
        thule_file_choice.layout.visibility = 'hidden'
        thule_grid_box.layout.visibility = 'hidden'
        thule_regrid.layout.visibility = 'hidden'
    else:
        summary_output.clear_output()
        job_output.clear_output()
        gen_opts.layout.visibility = 'hidden'
        regrid_choice.layout.visibility = 'hidden'
        is_choice.layout.visibility = 'hidden'
        grp_choice.layout.visibility = 'hidden'
        mod_choice.layout.visibility = 'hidden'
        exp_choice.layout.visibility = 'hidden'
        select_all.layout.visibility = 'hidden'
        file_selection.layout.visibility = 'hidden'
        exp_selection.layout.visibility = 'hidden'
        ingrid_choice.layout.visibility = 'hidden'
        outgrid_choice.layout.visibility = 'hidden'
        execute_regrid.layout.visibility = 'hidden'
        regrid_choice.value='File'
        #reveal thule widgets
        thule_instruction.layout.visibility = 'visible'
        check_for_files.layout.visibility = 'visible'
        
target_location = widgets.ToggleButtons(description='Target Location:',options=['CCR','Thule'], style = style)
target_location.value = 'CCR'
target_location.observe(target_change)

In [28]:
thule_instruction = widgets.Output(layout={'border': '1px solid black'})
with thule_instruction:
    print('Instruction for Regridding Thule Files:\n')
    print('1. Place your Thule Files in a Directory. ')
    print('   Make sure this directory is easily identifiable (name file using date, username, etc.)')
    print('2. Use Globus to Move your Directory into CCR Scratch Directory:')
    print('   /gpfs/scratch/ghub/regrid-tool/thule-regrid-targets')
    print('3. Hit the "Scan for Files" button to search for your files in scratch ')
    print('4. After the interface has been populated, select your directory from the dropdown widget')
    print('5. Select the files you wish to regrid by highlighting them with your cursor')
    print('6. Select the ingrid dimension of your files, and the outgrid dimension you wish to remap to')
    print('7. Hit the "Regrid Thule Files" button and await results')
    print('\nGlobus Endpoint Name: GHub-scratch-regrid-tool')
    print('Globus Website -- https://app.globus.org')
    print('Globus Help Docs -- https://docs.globus.org/how-to/get-started/')
thule_instruction.layout.visibility = 'hidden'

In [29]:
def check_thule_files(change):
    cmd = 'submit -v ccr-ghub -w 5 ./bin/check_thule.sh > term_out.txt'
    os.system(cmd)
    job_num = retrieve_job_num()
    rec_file = './'+job_num+'.stdout'
    f = open(rec_file,'r')
    of = open('thule_contents.txt','w')
    for line in f:
        of.write(line)
    f.close()
    of.close()
    move_to_data('thule_contents.txt')
    thule_dir_choice.options = thule_path_opts('')
    thule_dir_choice.layout.visibility = 'visible'
    thule_select_all.layout.visibility = 'visible'
    thule_file_choice.options = remove_scalars(thule_path_opts('/'+thule_dir_choice.value))
    thule_file_choice.layout.visibility = 'visible'
    thule_grid_box.layout.visibility = 'visible'
    thule_regrid.layout.visibility = 'visible'
    
check_for_files = widgets.Button(description = 'Scan for Files', button_style = 'info')
check_for_files.on_click(check_thule_files)
check_for_files.layout.visibility = 'hidden'

In [30]:
thule_path = '/gpfs/scratch/ghub/regrid-tool/thule-regrid-targets'
def thule_path_opts(path_ext):
    path = thule_path+path_ext
    f = open('/data/groups/ghub/tools/regrid/thule_contents.txt','r')
    lines = f.readlines()
    opts = []
    for i in range(0,len(lines)):
        if path == lines[i].rstrip():
            for j in range(i+1,len(lines)):
                if '/' in lines[j].rstrip():
                    break
                opts.append(lines[j].rstrip())
    return opts

def thule_dir_update(change):
    thule_select_all.value = False
    thule_file_choice.options = remove_scalars(thule_path_opts('/'+thule_dir_choice.value))
    
thule_dir_opts=['']
thule_dir_choice = widgets.Dropdown(description='Thule Directories: ', options = thule_dir_opts, style=style)
thule_dir_choice.observe(thule_dir_update)
thule_dir_choice.layout.visibility = 'hidden'
thule_file_opts=['']
thule_file_choice = widgets.SelectMultiple(description = 'Thule Files: ', options = thule_file_opts, layout = widgets.Layout(width='20%',height='100%'), style = style)
thule_file_choice.layout.visibility = 'hidden'

In [31]:
def thule_all_selected(change):
    if thule_select_all.value is True:
        thule_file_choice.value = thule_file_choice.options
        thule_file_choice.disabled = True
    else:
        thule_file_choice.value = []
        thule_file_choice.disabled = False
thule_select_all = widgets.Checkbox(value = False, description = 'Select All: ')
thule_select_all.observe(thule_all_selected)
thule_select_all.layout.visibility = 'hidden'

In [32]:
def thule_adjust_grid_opts(change):
    temp_list = list(thule_ingrid_choice.options)
    temp_list.remove(thule_ingrid_choice.value)
    thule_outgrid_choice.options = temp_list
    
grid_opts = ['0.5km','1km','2km','3km','4km','5km','6km','8km','10km','12km','15km','16km','20km','24km','30km','32km','40km','64km']
thule_ingrid_choice = widgets.Dropdown(description='Ingrid Dimension: ', options = grid_opts,style=style)
thule_ingrid_choice.observe(thule_adjust_grid_opts)
grid_opts.remove(thule_ingrid_choice.value)
thule_outgrid_choice = widgets.Dropdown(description='Outgrid Dimension: ', options = grid_opts,style=style)
thule_grid_box = HBox([thule_ingrid_choice,thule_outgrid_choice])
thule_grid_box.layout.visibility = 'hidden'

In [33]:
thule_regrid = widgets.Button(description='Regrid Thule Files',button_style='success')
thule_regrid.on_click(exe_thule_regrid)
thule_regrid.layout.visibility = 'hidden'

# User Interface

In [34]:
display(target_location)

ToggleButtons(description='Target Location:', options=('CCR', 'Thule'), style=ToggleButtonsStyle(description_w…

In [35]:
choice_box = HBox([grp_choice,mod_choice,exp_choice])
grid_choices = HBox([ingrid_choice,outgrid_choice])

In [36]:
display(gen_opts)
display(regrid_choice)
display(is_choice)
display(choice_box)
display(select_all)
display(exp_selection)
display(file_selection)
display(grid_choices)
display(execute_regrid)

Button(button_style='info', description='Update File Selection', style=ButtonStyle())

ToggleButtons(description='Regrid by: ', index=1, options=('Experiment', 'File'), value='File')

RadioButtons(description='Ice Sheet:', options=('Greenland', 'Antarctica'), style=DescriptionStyle(description…

Checkbox(value=False, description='Select All: ')

SelectMultiple(description='Experiments: ', layout=Layout(visibility='hidden'), options=(), style=DescriptionS…

SelectMultiple(description='netCDF Files:', layout=Layout(height='100%', visibility='visible', width='20%'), o…

Button(button_style='success', description='Regrid Files', style=ButtonStyle())

In [37]:
display(thule_instruction)
display(check_for_files)
display(thule_dir_choice)
display(thule_select_all)
display(thule_file_choice)
display(thule_grid_box)
display(thule_regrid)

Output(layout=Layout(border='1px solid black', visibility='hidden'))

Button(button_style='info', description='Scan for Files', layout=Layout(visibility='hidden'), style=ButtonStyl…

Dropdown(description='Thule Directories: ', layout=Layout(visibility='hidden'), options=('',), style=Descripti…

Checkbox(value=False, description='Select All: ', layout=Layout(visibility='hidden'))

SelectMultiple(description='Thule Files: ', layout=Layout(height='100%', visibility='hidden', width='20%'), op…

Button(button_style='success', description='Regrid Thule Files', layout=Layout(visibility='hidden'), style=But…

In [38]:
def show_job_info(change):
    if job_output.layout.visibility == 'visible':
        job_output.layout.visibility = 'hidden'
        showmore_button.description = 'Full Job Info'
    else:
        job_output.layout.visibility = 'visible'
        showmore_button.description = 'Less Job Info'
    
showmore_button.on_click(show_job_info)
job_output.layout.visibility = 'hidden'
display(summary_output)
display(showmore_button)
display(job_output)

Output(layout=Layout(border='1px solid black'))

Button(button_style='info', description='Full Job Info', style=ButtonStyle())

Output(layout=Layout(border='1px solid black', visibility='hidden'))